<a href="https://colab.research.google.com/github/tejaschaudhari2811/FIRE_Project_Task_1/blob/main/FIRE_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1 : Message Level Sentiment Analysis for Tamil Language

In [ ]:
import pandas as pd
import re


In [ ]:
print("Upload Training Data \n")
from google.colab import files
uploaded = files.upload()

Upload Training Data 



Saving tamil_offensive_train.tsv to tamil_offensive_train.tsv


In [ ]:
df = pd.read_table("tamil_offensive_train.tsv")

In [ ]:
# Define Polarities for numeric classification
polarities={'NOT':1, "OFF":-1,"not-Tamil":0}

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14130 entries, 0 to 14129
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  14130 non-null  int64 
 1   id          14130 non-null  object
 2   text        14130 non-null  object
 3   category    14130 non-null  object
dtypes: int64(1), object(3)
memory usage: 441.7+ KB
None


In [ ]:
df.groupby(by=['category']).count()

,Unnamed: 0,id,text
category,,,
NOT,11357,11357,11357
OFF,2766,2766,2766
not-Tamil,7,7,7


In [ ]:
# Supplementary Functions
def convert_polarity(category):
    """
    Takes in category as string 
    outputs integer representation
    """
    return polarities[category]

def word_length(sentence):
  """
  Takes in Tamil Text
  Counts Words
  Returns number of Words
  """
  return len(sentence.split(" "))

In [ ]:
def dataframe_modification(df):
  df['polarity'] = df['category'].apply(convert_polarity)
  df['text_length'] = df['text'].apply(len)
  df['number_of_words'] = df['text'].apply(word_length)
  return df

In [ ]:
df.head()

,Unnamed: 0,id,text,category
0,0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [ ]:
df = df.drop(df[df.polarity == 0].index)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [ ]:
tfidf = TfidfVectorizer(max_features=1000)

In [ ]:
!pip install imbalanced-learn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()

In [ ]:
# Get the data from Dataframe.
X = df['text']
y = df['polarity']

# Convert data to vectors
X = tfidf.fit_transform(X)
X_ros,y_ros = ros.fit_resample(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def svm_classifier(X_ros,y_ros):
  """
  Takes in oversampled data
  Outputs support vector machine model and classification report
  """
  X_train,X_test,y_train,y_test = train_test_split(X_ros,y_ros,test_size=0.2, random_state=0)
  clf = LinearSVC()
  linear_svc_model = clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  report = classification_report(y_pred,y_test)
  return linear_svc_model,report


In [ ]:
model,report= svm_classifier(X_ros,y_ros)

In [ ]:
print(report)

              precision    recall  f1-score   support

          -1       0.83      0.76      0.79      2499
           1       0.73      0.82      0.77      2044

    accuracy                           0.78      4543
   macro avg       0.78      0.79      0.78      4543
weighted avg       0.79      0.78      0.78      4543



# Using Logistic Regression for Classification

In [ ]:
def logistic_regression_classifier(X_ros,y_ros):
  """
  Takes in oversampled data
  Outputs Linear Regression Model and classification report
  """
  X_train,X_test,y_train,y_test = train_test_split(X_ros,y_ros,test_size=0.2, random_state=0)
  clf = LogisticRegression(random_state=0)
  logistic_regression_model = clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  report = classification_report(y_pred,y_test)
  return logistic_regression_model,report

In [ ]:
lr_model,report = logistic_regression_classifier(X_ros,y_ros)
print(report)

              precision    recall  f1-score   support

          -1       0.80      0.74      0.77      2401
           0       1.00      1.00      1.00      2362
           1       0.72      0.79      0.75      2052

    accuracy                           0.84      6815
   macro avg       0.84      0.84      0.84      6815
weighted avg       0.85      0.84      0.85      6815



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Use of Indic-nlp-library for Text Processing

In [ ]:
!git clone "https://github.com/anoopkunchukuttan/indic_nlp_resources.git"

In [ ]:
!pip install indic-nlp-library

In [ ]:
INDIC_NLP_RESOURCES=r"/content/indic_nlp_resources"

In [ ]:
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)

In [ ]:
from indicnlp import loader
loader.load()

In [ ]:
from indicnlp.tokenize import sentence_tokenize, indic_tokenize
from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator, ItransTransliterator
string = "திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்"
sentences = sentence_tokenize.sentence_split(string, lang='ta')
for se in indic_tokenize.trivial_tokenize(string):
  print(ItransTransliterator.to_itrans(se,'en'))

திருமலை
நாயக்கர்
பேரவை
சார்பாக
படம்
வெற்றி
பெற
வாழ்த்துக்கள்


In [ ]:
from indicnlp.transliterate.unicode_transliterate import ItransTransliterator

#input_text='राजस्थान'
input_text='திருமலை'
lang='ta'

print(ItransTransliterator.to_itrans(input_text,lang))

tirumalai


In [ ]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = TFBertModel.from_pretrained("bert-base-multilingual-uncased")


Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
text = "திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

In [ ]:
X_hf = df['text']
y_hf = df['polarity']

X_hf = X_hf.apply(tokenizer)

In [ ]:
X_hf[0]

{'input_ids': [101, 54552, 60211, 25088, 808, 37025, 20798, 13047, 810, 55541, 20841, 802, 15241, 85286, 12146, 810, 44042, 818, 96902, 49815, 810, 96902, 36718, 818, 13744, 40806, 95588, 12076, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
import numpy as np

In [ ]:
# Remove Unnecessary keys and values from the embeddings
X_train = []
for text in X_hf:
  text.pop('token_type_ids')
  text.pop('attention_mask')
  X_train.append(np.array(text['input_ids']))

print("Finished")

Finished


In [ ]:
X_ros,y_ros = ros.fit_resample(X_train,y_hf)

# Get Emoji Embeddings

In [ ]:
!pip install gensim

In [ ]:
!git clone "https://github.com/uclnlp/emoji2vec.git"

Cloning into 'emoji2vec'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 62 (delta 0), reused 1 (delta 0), pack-reused 59
Unpacking objects: 100% (62/62), done.


In [ ]:
import gensim.models as gsm

e2v = gsm.KeyedVectors.load_word2vec_format('/content/emoji2vec/pre-trained/emoji2vec.bin', binary=True)
happy_vector = e2v['😂']    # Produces an embedding vector of length 300

In [ ]:
print(happy_vector)

[ 0.00368995 -0.00747812  0.04797658  0.0592004  -0.0594517  -0.01919357
  0.06628808 -0.06752758  0.10211562  0.087887   -0.02949324 -0.07718539
 -0.07068242  0.04688684 -0.07970048  0.09425059  0.06230384  0.09221437
 -0.03019106 -0.08475313  0.06153254  0.08948752  0.07492744 -0.0697182
  0.02842957 -0.07775956 -0.00585377 -0.01818646  0.07633407 -0.09874155
 -0.05507204  0.04539224  0.00842756 -0.06562244 -0.04902945 -0.05832005
  0.07070394  0.05070673  0.05466724  0.08123799  0.06764334 -0.09264724
  0.07862549 -0.03581576  0.02264971 -0.00622183 -0.0318586   0.0421854
 -0.05610641  0.07420981 -0.06671992 -0.05842454  0.03449175  0.07620545
  0.08768762  0.03041447 -0.05500258 -0.03190211  0.07291865 -0.04514114
 -0.0541351   0.05987531 -0.0346354  -0.01317827  0.00849659 -0.01372368
 -0.01279207 -0.01283779 -0.07702036  0.09149366  0.0390107   0.07108027
 -0.01238731  0.08397982 -0.08521689 -0.06615324  0.02603729  0.07916152
  0.08146766  0.09780202 -0.05352468  0.03703088  0.0

# Dataset Expansion using INDIC NLP Romanization Module

In [ ]:
from indicnlp.transliterate.unicode_transliterate import ItransTransliterator

input_text='திருமலை'
lang='ta'

print(ItransTransliterator.to_itrans(input_text,lang))

tirumalai


In [ ]:
English_Tamil_comments = []
id = 0
id_list = []

for text in df['text']:
  id_list.append(str("eng{}".format(id)))
  English_Tamil_comments.append((ItransTransliterator.to_itrans(text,'ta')))
  id +=1

In [ ]:
dict_ = {'id':id_list,'text':English_Tamil_comments, 'category':df['category']}

In [ ]:
new_df = pd.DataFrame.from_dict(dict_)
df = df.append(new_df,ignore_index=True)

# Add English Translated Comments to Data

In [ ]:
!pip install googletrans==3.1.0a0

Processing /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac/googletrans-3.1.0a0-cp37-none-any.whl
ERROR: Operation cancelled by user


In [ ]:
!pip uninstall googletrans

Uninstalling googletrans-3.0.0:
  Would remove:
    /usr/local/bin/translate
    /usr/local/lib/python3.7/dist-packages/googletrans-3.0.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/googletrans/*
Proceed (y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/uninstall.py", line 79, in run
    auto_confirm=options.yes, verbose=self.verbosity > 0,
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_install.py", line 755, in uninstall
    uninstalled_pathset.remove(auto_confirm, verbose)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_uninstall.py", line 388, in remove
    if auto_confirm or self._allowed_to_proceed(verbose):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/req/req_uninstall.py", line 

In [ ]:
from googletrans import Translator, constants
translator = Translator()

In [ ]:
translation = translator.translate("திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்")
print(translation.text)

AttributeError: ignored

In [ ]:
Tamil_translated_comments = []
id = 0
id_list = []

for text in df['text']:
  id_list.append(str("trans_{}".format(id)))
  id +=1
  if id < 2370:
    continue
  translator.raise_Exception = True
  translation = translator.translate(text)
  print(id, translation.text)
  Tamil_translated_comments.append(translation.text)
  
  
print(Tamil_translated_comments)

In [ ]:
translations = translator.translate(df['text'].values.tolist())

In [ ]:
import pickle
with open("tamil_english_translated_2.pkl","wb") as f:
  pickle.dump(Tamil_translated_comments,f)

# Convert Translated Comments to Dataframe

In [ ]:
import pickle
with open("/content/tamil_english_translated_1.pkl","rb") as f:
  list_1 = pickle.load(f)

In [ ]:
len(list_1)

2370

In [ ]:
translation_category_list = df['category'].values.tolist()

In [ ]:
translation_category_list = translation_category_list[:2370]

In [ ]:
trans_id_list=[]
for i in range(1,2371,1):
  id = 'trans'+str(i)
  trans_id_list.append(id)

In [ ]:
dict_ = {'id':trans_id_list,'text':list_1, 'category':translation_category_list}

In [ ]:
new_df = pd.DataFrame.from_dict(dict_)

In [ ]:
df = pd.read_csv('romanized_tamil_data_added.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11760 entries, 0 to 11759
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  11760 non-null  int64 
 1   id          11760 non-null  object
 2   text        11760 non-null  object
 3   category    11760 non-null  object
dtypes: int64(1), object(3)
memory usage: 367.6+ KB


In [ ]:
df=df.append(new_df, ignore_index=True)
df.to_csv('romanized_translated_data.csv')

In [ ]:
#dict_ = {'id':id_list,'text':Tamil_translated_comments, 'category':df['category']}
dict_ = {text':Tamil_translated_comments}

In [ ]:
print(Tamil_translated_comments)

In [ ]:
# Start translation again from 2370

In [ ]:
new_df = pd.DataFrame.from_dict(dict_)
#df = df.append(new_df,ignore_index=True)

# Create Embeddings Dictionary
### 1. Find all unique emojis in the dataset
### 2. Get embeddings for the emojis
### 3. Convert emojis to Embeddings and replace in senteces after embeddings.

In [106]:
!pip install emojis

In [107]:
# Check emojis in each sentence, if present, extract out.
import emojis

emoji_list = []

for text in df['text']:

  set_ = emojis.get(text)
  
  if len(set_) != 0:
    emoji_list.append(list(set_))

In [108]:
# Create list of Unique Emojis
unique_emojis = []

for emoji in emoji_list:
  
  for emo in emoji:
    
    if emo not in unique_emojis:
      unique_emojis.append(emo)

In [ ]:
for emoji in unique_emojis:
  try:
    (e2v[emoji])
  except:
    KeyError
    print(emoji)

In [114]:
!pip install emoji

In [119]:
emoji_dict={}
import emoji
for emo in unique_emojis:
  emoji_dict[emo]= emoji.demojize(str(emo))

print(emoji_dict)

{'🦁': ':lion:', '🤣': ':rolling_on_the_floor_laughing:', '🤦': ':person_facepalming:', '🤔': ':thinking_face:', '🤘': ':sign_of_the_horns:', '🤫': ':shushing_face:', '🤩': ':star-struck:', '🤝': ':handshake:', '🥭': ':mango:', '🥇': ':1st_place_medal:', '🤞': ':crossed_fingers:', '🤕': ':face_with_head-bandage:', '🤭': ':face_with_hand_over_mouth:', '🥶': ':cold_face:', '🥳': ':partying_face:', '🤪': ':zany_face:', '🤗': ':hugging_face:', '🤬': ':face_with_symbols_on_mouth:', '🤟': ':love-you_gesture:', '🤧': ':sneezing_face:', '\U0001f7e0': ':orange_circle:', '🥊': ':boxing_glove:', '🦍': ':gorilla:', '🤐': ':zipper-mouth_face:', '🤙': ':call_me_hand:', '🧟': ':zombie:', '🥁': ':drum:', '🤺': ':person_fencing:', '🤨': ':face_with_raised_eyebrow:', '🤠': ':cowboy_hat_face:', '🦄': ':unicorn:', '🦌': ':deer:', '🧡': ':orange_heart:', '🧐': ':face_with_monocle:', '🤛': ':left-facing_fist:', '🦸': ':superhero:', '🤓': ':nerd_face:', '🧨': ':firecracker:', '🥰': ':smiling_face_with_hearts:', '🤮': ':face_vomiting:', '⏰': ':ala

# BERT_Multilingual_Embeddings

In [ ]:
!pip install transformers

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertModel

In [ ]:
from sklearn.model_selection import train_test_split

import pandas as pd

In [ ]:
df = pd.read_table("tamil_offensive_train.tsv")

train_df, test_df = train_test_split(df,test_size=0.2,random_state=32)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

In [ ]:
features =[]

In [ ]:
model = BertModel.from_pretrained("bert-base-multilingual-uncased")

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_texts = train_df['text'].values.tolist()
train_targets = train_df['category'].values.tolist()
train_targets = [ 1  if t == 'OFF' else 0 for t in train_targets]

train_df['num_words'] = train_df['text'].str.split().apply(len)

train_df.num_words.quantile(0.98)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


52.0

In [ ]:
sample_text = train_texts[:64]
#need to run this in batches to extract feature for entire dataset
t = tokenizer(sample_text,max_length = 128,
                                padding = "max_length",
                                truncation = True)

In [ ]:
input_ids = torch.tensor(t['input_ids'])
attention_masks = torch.tensor(t['attention_mask'])

In [ ]:
with torch.no_grad():
    lhs,pooler_output = model(input_ids,attention_masks,return_dict=False)

In [ ]:
print(sample_text[0])

நாயுடு இன மக்கள் சார்பாக இப்படம் வெற்றி பெற வாழ்த்துக்கள்


In [ ]:
pooler_output

tensor([[ 0.2038,  0.0239,  0.1689,  ...,  0.0469, -0.1777, -0.1806],
        [ 0.1562,  0.0432,  0.1683,  ...,  0.0359, -0.1625, -0.1776],
        [-0.0627,  0.2241,  0.0246,  ..., -0.0363, -0.1432, -0.0345],
        ...,
        [ 0.0320,  0.1387,  0.0961,  ...,  0.0311, -0.1818, -0.1429],
        [ 0.2781,  0.0265,  0.2385,  ...,  0.0797, -0.1620, -0.2133],
        [ 0.1049,  0.0713,  0.1367,  ...,  0.0635, -0.2036, -0.1711]])

In [ ]:
id = 0
f = open("not_translated.txt","w")

for text in df['text']:
  
  id +=1

  if id < 2370:
    continue
  
  f.write(text+"|"+"\n")
  
  
f.close()

In [ ]:
import json
import requests
from urllib.parse import  quote

text=quote('திருமலை')
# text=quote('मनिश् जोए')
url='http://www.cfilt.iitb.ac.in/indicnlpweb/indicnlpws/translate/ta/en/{}/'.format(text)
## Note the forward slash '/' at the end of the URL. It's should be there, but please live with it for now!

response = requests.get(url)
response.json()['en']

'Thirumalai '

In [ ]:
Tamil_translated_comments = []
id = 0
id_list = []

for text in df['text']:
  id_list.append(str("trans_{}".format(id)))
  id +=1
  text=quote(text)
  if id < 2370:
    continue
  url='http://www.cfilt.iitb.ac.in/indicnlpweb/indicnlpws/translate/ta/en/{}/'.format(text)
  response = requests.get(url)
  print(response.json()['en'])
  
  
print(Tamil_translated_comments)

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 19.8MB/s 
     |████████████████████████████████| 3.3MB 35.8MB/s 
     |████████████████████████████████| 901kB 40.3MB/s 


In [ ]:
en_ta_translator=pipeline(task='translation_ta_to_en',model='Helsinki-NLP/opus-mt-mul-en')

# Use of Soundex Algorithm
1. Find all unique words in the dataset
2. Compare Each word to another in set of two
3. Make a dictionary with key as soundex code and values as scores with either 1 or 2
4. Create another data copy and replace each word by Soundex code
5. Train classifier on this new data

In [ ]:
!pip install libindic-soundex

  Created wheel for libindic-soundex: filename=libindic_soundex-1.0.2-cp37-none-any.whl size=10013 sha256=96a4d9e97778f987ace0d84b6f5ffe575cf75ae8d31bc0b67bf0fa4ca92b22fa
  Stored in directory: /root/.cache/pip/wheels/db/c3/71/9229ab2a81f9b183cb804829e29a247d18dafe637db8ccf25f
Successfully built libindic-soundex


In [ ]:
!pip install libindic-utils>=1.0.3

## Task 1 : Find all unique words in the dataset.

In [ ]:
df = pd.read_csv("/content/romanized_translated_data.csv")

In [ ]:
df= dataframe_modification(df)
df.head()

,id,text,category,polarity,text_length,number_of_words
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT,1,60,8
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF,-1,80,12
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT,1,69,7
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF,-1,84,13
4,tam5,only for விஜய் சேதுபதி and STR,NOT,1,30,6


In [ ]:
texts = df['text'].values.tolist()

In [ ]:
unique_words = []
for text in texts:
  for w in text.split():
    if w not in unique_words:
      unique_words.append(w)

In [ ]:
len(unique_words)

54938

In [ ]:
from libindic.soundex import Soundex
instance = Soundex()
print(instance.soundex('tirumalai'))
instance.compare('ಬೆಂಗಳೂರು','बॆंगळूरु')

t654


2

In [117]:
# find the pairs of same sounding words
count = 0
for i in range(len(unique_words)):
  word_1 = str(unique_words[i])
  for j in range((len(unique_words))):
    if i != j:
      word_2 = str(unique_words[j])
      if instance.compare(word_1,word_2) == 1 or instance.compare(word_1,word_2):
        count +=1


print(count)

KeyboardInterrupt: ignored

In [118]:
print(count)

51376445


In [ ]:
list_1 = df['text'][:5880].values.tolist()
list_2 = df['text'][5880:11761].values.tolist()

In [ ]:
# Get the data from Dataframe.
X = df['text']
y = df['polarity']

# Convert data to vectors
X = tfidf.fit_transform(X)
X_ros,y_ros = ros.fit_resample(X,y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
model,report = logistic_regression_classifier(X_ros,y_ros)
print(report)

              precision    recall  f1-score   support

          -1       0.82      0.75      0.78      2464
           1       0.73      0.80      0.76      2079

    accuracy                           0.77      4543
   macro avg       0.77      0.77      0.77      4543
weighted avg       0.78      0.77      0.77      4543



In [ ]:
soundex_codes={}
for word in unique_words:
  soundex_codes[word] = instance.soundex(word)

print(soundex_codes)

{'திருமலை': 'தBPCNQD0', 'நாயக்கர்': 'நAOFP000', 'பேரவை': 'பDPRD000', 'சார்பாக': 'சAPMAF00', 'படம்': 'பIN00000', 'வெற்றி': 'வDPB0000', 'பெற': 'பDP00000', 'வாழ்த்துக்கள்': 'வAQKCFQ0', 'இந்த': 'இLK00000', 'ட்ரெய்லர்': 'டPDOQP00', 'கூட': 'கCI00000', 'பார்க்கிற': 'பAPFBP00', 'மாதிரி': 'மAKBPB00', 'இல்லை..': 'இQD00000', 'இதை': 'இKD00000', 'தியேட்டர்': 'தBODIP00', 'ல': 'ல0000000', 'போய்': 'பEO00000', 'பார்க்கனுமா..': 'பAPFLCNA', '.': '.0000000', 'மைசூரு': 'மDHCPC00', 'செட்டியார்': 'சDIBOAP0', 'சமூகத்தின்': 'சNCFKBL0', 'இப்படம்': 'இMIN0000', 'வெற்றிபெற': 'வDPBMDP0', 'வாழ்த்துகிறேன்': 'வAQKCFBP', 'மொத்த': 'மEK00000', 'சாதியும்': 'சAKBOCN0', 'ஒரு': 'ஒPC00000', 'சாதிக்கு': 'சAKBFC00', 'எதிரா': 'எKBPA000', 'நிக்குது......': 'நBFCKC00', 'என்னா': 'எLA00000', 'சமூகம்': 'சNCFN000', 'டா': 'டA000000', 'இது': 'இKC00000', 'தூ...........': 'தC000000', 'only': 'o54', 'for': 'f6', 'விஜய்': 'வBHO0000', 'சேதுபதி': 'சDKCMKB0', 'and': 'a53', 'STR': 'S36', 'உணமையாவே': 'உJNDOARD', 'சைக்கோ': 'சDFE0000', 'தான்': 'தA